In [1]:
import numpy as np
import pandas as pd
from logistic_regression import logistic_regression  

data = pd.read_csv('COL774_A5_ODT/train_sample.csv',header=None)
X = data.drop(data.columns[-1], axis=1).values
Y = data[data.columns[-1]].values
print(X.shape)
print(Y.shape)

(800, 2)
(800,)


In [2]:
import pandas as pd
import numpy as np
import csv  # Import the csv module

class ObliqueDecisionTree:
    def __init__(self, max_depth, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None
        self.node_info = []  # To store node information for CSV

    class Node:
        def __init__(self, depth=0, node_id=None):
            self.depth = depth
            self.node_id = node_id  
            self.is_leaf = False
            self.split_value = None
            self.feature_combination = None
            self.left = None
            self.right = None
            self.prediction = None
            self.weights = None 

    def _custom_logistic_regression_split(self, X, y):
        feature_combination = logistic_regression(X, y)  
        linear_combination = X.dot(feature_combination)
        return linear_combination, feature_combination

    def _gini_impurity(self, y):
        m = len(y)
        if m == 0:
            return 0
        p1 = np.sum(y) / m
        p0 = 1 - p1
        return 1 - p1**2 - p0**2

    def _best_threshold(self, linear_combination, y):
        sort_idx = np.argsort(linear_combination)
        sorted_combination = linear_combination[sort_idx]
        sorted_y = y[sort_idx]
        
        best_gini = float('inf')
        best_threshold = None
        n = len(y)

        for i in range(1, n):
            left_y, right_y = sorted_y[:i], sorted_y[i:]
            gini_left = self._gini_impurity(left_y)
            gini_right = self._gini_impurity(right_y)
            weighted_gini = (len(left_y) * gini_left + len(right_y) * gini_right) / n
            
            if weighted_gini < best_gini and sorted_combination[i-1]!=sorted_combination[i]:
                best_gini = weighted_gini
                best_threshold = (sorted_combination[i - 1] + sorted_combination[i]) / 2
                
        return best_threshold, best_gini

    def _build_tree(self, X, y, depth=0, node_id=1):
        node = self.Node(depth, node_id)
        
        if (self.max_depth and depth >= self.max_depth) or len(y) < self.min_samples_split or np.all(y == y[0]):
            node.is_leaf = True
            node.prediction = np.round(np.mean(y)).astype(int)
            return node

        linear_combination, feature_combination = self._custom_logistic_regression_split(X, y)
        best_threshold, best_gini = self._best_threshold(linear_combination, y)

        if best_gini == 1.0:
            node.is_leaf = True
            node.prediction = np.round(np.mean(y)).astype(int)
            return node

        left_mask = linear_combination <= best_threshold
        right_mask = linear_combination > best_threshold
        
        node.split_value = best_threshold
        node.feature_combination = feature_combination
        node.weights = feature_combination  # Store as an array

        self.node_info.append([node.node_id, node.weights, node.split_value])

        node.left = self._build_tree(X[left_mask], y[left_mask], depth + 1, 2 * node_id)  # Left child
        node.right = self._build_tree(X[right_mask], y[right_mask], depth + 1, 2 * node_id + 1)  # Right child
        
        return node

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_sample(self, x, node):
        if node.is_leaf:
            return node.prediction
        linear_combination = np.dot(x, node.feature_combination)
        if linear_combination <= node.split_value:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

    def predict(self, X):
        return np.array([self._predict_sample(x, self.tree) for x in X])

    def save_weights_to_csv(self, filename):
        
        formatted_node_info = []
        for node_id, weights, threshold in self.node_info:
            weight_list = weights.flatten().tolist() 
            formatted_node_info.append([node_id] + weight_list + [threshold])  
       
        df = pd.DataFrame(formatted_node_info)
        df.to_csv(filename, index=False, header=False)


tree = ObliqueDecisionTree(max_depth=11)
tree.fit(X, Y)
tree.save_weights_to_csv('weights.csv')
